In [1]:
import folium
import sqlite3

In [2]:
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

# Create the users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    preferences TEXT
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

# Create the users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS attractions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(255),
    category VARCHAR(50),
    description TEXT,
    city VARCHAR(100)
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [4]:
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

# Check if column exists
cursor.execute("PRAGMA table_info(users)")
columns = [column[1] for column in cursor.fetchall()]

# Add column only if it doesn't exist
if 'hometown' not in columns:
    cursor.execute('''
    ALTER TABLE users ADD COLUMN hometown TEXT
    ''')
    print("Added 'hometown' column to users table")
else:
    print("Column 'hometown' already exists")

# Commit the changes and close the connection
conn.commit()
conn.close()

Column 'hometown' already exists


In [5]:
# Example of inserting a user with preferences
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

sample_preferences = "I love skiing and I live in New York City"

# First check if user exists
cursor.execute('SELECT username FROM users WHERE username = ?', ('Sam Controlman',))
if not cursor.fetchone():
    cursor.execute('''
    INSERT INTO users (username, preferences) 
    VALUES (?, ?)
    ''', ('Sam Controlman', sample_preferences))

conn.commit()
conn.close()

In [6]:
# Example of inserting a user with preferences
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

sample_preferences = "I love skiing and I live in New York City"

cursor.execute('''
UPDATE users SET hometown = (?) WHERE username = 'Sam Controlman'
''', ('New York City',))

conn.commit()
conn.close()

In [7]:
# Example of querying the data
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM users')
users = cursor.fetchall()
for user in users:
    print(f"User ID: {user[0]}")
    print(f"Username: {user[1]}")
    print(f"Preferences: {user[2]}")
    print(f"Hometown: {user[3]}\n")

conn.close()

User ID: 1
Username: Sam Controlman
Preferences: skiing
Hometown: New York City

User ID: 2
Username: sam controlman
Preferences: skiing
Hometown: None

User ID: 4
Username: sam
Preferences: swimming
Hometown: None

User ID: 9
Username: Sam Altman
Preferences: swimming
Hometown: None

User ID: 12
Username: Sam Norman
Preferences: luxury
Hometown: None



In [8]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import sent_tokenize
import sqlite3
import pickle
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Initialize the sentence transformer model
encoding_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_chunking(text: str, chunk_size: int = 500, chunk_overlap: int = 50) -> List[str]:
    """Split text into semantic chunks using LangChain's RecursiveCharacterTextSplitter"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    
    return text_splitter.split_text(text)

class CityVectorDB:
    def __init__(self):
        self.dimension = 384  # dimension of all-MiniLM-L6-v2 embeddings
        self.index = faiss.IndexFlatL2(self.dimension)
        self.cities_data = []
        self.chunks_metadata = []
    
    def add_city(self, city: str, description: str):
        # Create chunks
        chunks = semantic_chunking(description)
        
        # Create embeddings for chunks
        embeddings = encoding_model.encode(chunks)
        
        # Add to FAISS index
        self.index.add(np.array(embeddings).astype('float32'))
        
        # Store metadata
        for i, chunk in enumerate(chunks):
            self.chunks_metadata.append({
                'city': city,
                'chunk': chunk,
                'chunk_id': len(self.chunks_metadata) + i
            })
    
    def search(self, query: str, k: int = 3) -> List[Dict]:
        # Create query embedding
        query_vector = encoding_model.encode([query])
        
        # Search in FAISS
        distances, indices = self.index.search(query_vector.astype('float32'), k)
        
        # Get results
        results = []
        for idx in indices[0]:
            if idx != -1:  # FAISS returns -1 if there aren't enough results
                results.append(self.chunks_metadata[idx])
        
        return results
    
    def save(self, filename: str):
        # Save FAISS index
        faiss.write_index(self.index, f"{filename}.index")
        # Save metadata
        with open(f"{filename}.metadata", 'wb') as f:
            pickle.dump(self.chunks_metadata, f)
    
    def load(self, filename: str):
        # Load FAISS index
        self.index = faiss.read_index(f"{filename}.index")
        # Load metadata
        with open(f"{filename}.metadata", 'rb') as f:
            self.chunks_metadata = pickle.load(f)

In [10]:
# Initialize and populate the vector database
cities_data = [
    {
        "city": "New York City, USA",
        "description": """New York City is perfect for urban explorers and culture enthusiasts. 
        It offers world-class museums, Broadway shows, and diverse neighborhoods. 
        The city is known for its shopping, fine dining, and vibrant nightlife. 
        Central Park provides green space for outdoor activities. 
        It's ideal for those who love fast-paced city life and cultural experiences."""
    },
    {
        "city": "Aspen, Colorado, USA",
        "description": """Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers."""
    },
    {
        "city": "Kyoto, Japan",
        "description": """Kyoto is a city steeped in traditional Japanese culture and history.
        It features numerous ancient temples, beautiful gardens, and traditional tea houses.
        The city is famous for its cherry blossoms in spring and vibrant autumn colors.
        Visitors can experience authentic Japanese cuisine and traditional arts.
        It's perfect for those interested in cultural immersion and peaceful environments."""
    },
    {
        "city": "Paris, France",
        "description": """Paris is a city of romance and sophistication. 
        It is renowned for its iconic landmarks such as the Eiffel Tower and Louvre Museum. 
        Visitors can enjoy world-class art, haute cuisine, and chic shopping. 
        The Seine River and charming cafes create a picturesque setting. 
        It's ideal for those seeking romance, art, and gastronomy."""
    },
    {
        "city": "Sydney, Australia",
        "description": """Sydney offers a perfect mix of urban life and natural beauty. 
        It's famous for its iconic Sydney Opera House and Harbour Bridge. 
        Visitors can enjoy beautiful beaches like Bondi and Manly. 
        The city also has vibrant markets, fine dining, and wildlife parks. 
        It's perfect for city explorers and beach lovers."""
    },
    {
        "city": "Cape Town, South Africa",
        "description": """Cape Town is known for its stunning landscapes and cultural diversity. 
        It boasts Table Mountain, beautiful beaches, and vibrant neighborhoods. 
        Visitors can explore vineyards, enjoy wildlife safaris, and experience local cuisine. 
        The city's rich history is evident in sites like Robben Island. 
        It's ideal for adventure seekers and history enthusiasts."""
    },
    {
        "city": "Barcelona, Spain",
        "description": """Barcelona combines unique architecture, rich culture, and Mediterranean charm. 
        It features Gaudi's masterpieces like the Sagrada Familia and Park Güell. 
        Visitors can enjoy bustling markets, beautiful beaches, and vibrant nightlife. 
        The city's Catalan cuisine is a culinary delight. 
        It's perfect for architecture buffs and cultural explorers."""
    },
    {
        "city": "Reykjavik, Iceland",
        "description": """Reykjavik is a gateway to Iceland's natural wonders. 
        It offers unique experiences like the Northern Lights and geothermal lagoons. 
        Visitors can explore volcanic landscapes, waterfalls, and glaciers. 
        The city itself has a vibrant arts scene and welcoming atmosphere. 
        It's ideal for nature lovers and adventure travelers."""
    },
    {
        "city": "Santorini, Greece",
        "description": """Santorini is known for its stunning sunsets and whitewashed buildings. 
        It offers breathtaking views of the Aegean Sea and charming villages. 
        Visitors can explore ancient ruins, volcanic beaches, and enjoy local wines. 
        The island's romantic ambiance makes it a favorite for couples. 
        It's perfect for romantic getaways and scenic relaxation."""
    },
    {
        "city": "Tokyo, Japan",
        "description": """Tokyo is a dynamic city blending tradition with innovation. 
        It offers futuristic skyscrapers, historic temples, and bustling markets. 
        Visitors can enjoy diverse cuisines, vibrant shopping districts, and unique cultural experiences. 
        The city's efficient public transport makes it easy to explore. 
        It's ideal for tech enthusiasts and cultural adventurers."""
    },
    {
        "city": "Rome, Italy",
        "description": """Rome is a treasure trove of history and art. 
        The city is home to ancient landmarks like the Colosseum, Roman Forum, and the Pantheon. 
        Visitors can explore the Vatican City, Sistine Chapel, and St. Peter's Basilica. 
        Rome's vibrant streets are filled with charming cafes and exquisite Italian cuisine. 
        It's perfect for history buffs and lovers of fine food."""
    },
    {
        "city": "Banff, Canada",
        "description": """Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers."""
    },
    {
        "city": "Buenos Aires, Argentina",
        "description": """Buenos Aires is a vibrant city known for its rich culture and lively atmosphere. 
        It features colorful neighborhoods like La Boca, elegant architecture, and historic sites. 
        Visitors can experience tango dancing, world-class steaks, and bustling markets. 
        The city's blend of European and Latin American influences is unique. 
        It's perfect for cultural explorers and food enthusiasts."""
    },
    {
        "city": "Dubai, UAE",
        "description": """Dubai is a futuristic city known for its luxury and innovation. 
        It features iconic landmarks like the Burj Khalifa, Palm Jumeirah, and Dubai Mall. 
        Visitors can enjoy desert safaris, luxury shopping, and world-class dining. 
        The city also offers cultural experiences like traditional souks and dhow cruises. 
        It's ideal for luxury travelers and adventure seekers."""
    },
    {
        "city": "Queenstown, New Zealand",
        "description": """Queenstown is an adventure capital surrounded by stunning natural beauty. 
        It offers activities like bungee jumping, skydiving, and jet boating. 
        Visitors can explore Lake Wakatipu, nearby wineries, and mountain trails. 
        The town's vibrant atmosphere includes cozy cafes and lively bars. 
        It's perfect for thrill-seekers and nature enthusiasts."""
    }
]

In [11]:
# Initialize the vector database
vector_db = CityVectorDB()

# Add cities to the database
for city_data in cities_data:
    vector_db.add_city(city_data["city"], city_data["description"])

# Save the database
vector_db.save("cities_vector_db")

Function to be used in the tool

In [12]:
def get_city_recommendations(username: str, k: int = 3) -> List[Dict]:
    # Connect to SQLite database
    conn = sqlite3.connect('user_preferences.db')
    cursor = conn.cursor()
    
    # Get user preferences
    cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
    result = cursor.fetchone()
    conn.close()
    
    if result:
        user_preferences = result[0]
        # Find matching cities
        matches = vector_db.search(user_preferences, k)
        return matches
    
    return []

In [13]:
# Test the system
test_results = get_city_recommendations('Sam Controlman')
print("\nRecommended Cities based on your preferences:")
for result in test_results:
    print(f"\nCity: {result['city']}")
    print(f"Matching Description: {result['chunk']}")


Recommended Cities based on your preferences:

City: Aspen, Colorado, USA
Matching Description: Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers.

City: Banff, Canada
Matching Description: Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers.

City: Queenstown, New Zealand
Matching Description: Queenstown is an adventure capital surrounded by st

In [14]:
from langchain.tools import BaseTool
from pydantic import Field

class CityRecommendationTool(BaseTool):
    name: str = "city_recommendation_tool"
    description: str = """
    Gets personalized city recommendations based on a user's stored preferences.
    Input should be a username (string) and optionally the number of recommendations (integer).
    Should be called only when we do not know the user's travel destination and not multiple times.
    """
    
    vector_db: CityVectorDB = Field(
        default_factory=CityVectorDB,
        description="Vector database for city matching"
    )
    model: SentenceTransformer = Field(
        default_factory=lambda: SentenceTransformer('all-MiniLM-L6-v2'),
        description="Sentence transformer model for encoding"
    )
    
    def __init__(self, **data):
        super().__init__(**data)
        self.vector_db.load("cities_vector_db")  # Load the saved vector database
    
    def _run(self, username: str, k: int = 3) -> str:
        """Get city recommendations for a user based on their stored preferences"""
        try:
            # Connect to database
            with sqlite3.connect('user_preferences.db') as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
                result = cursor.fetchone()
            
            if not result:
                return f"No preferences found for user: {username}. Please store preferences first."
            
            # Get recommendations using the vector_db's search method
            user_preferences = result[0]
            matches = self.vector_db.search(user_preferences, k)
            
            if not matches:
                return "No matching cities found based on your preferences."
            
            # Format response
            response = [f"Based on your preferences: '{user_preferences}'\n"]
            response.append(f"Here are {len(matches)} recommended cities:\n")
            
            for i, match in enumerate(matches, 1):
                response.append(f"{i}. {match['city']}")
                response.append(f"   Matching aspects: {match['chunk']}\n")
            
            return "\n".join(response)
            
        except Exception as e:
            return f"Error getting recommendations: {str(e)}"
    
    def _arun(self, username: str, k: int = 3) -> str:
        """Async version - Not implemented"""
        raise NotImplementedError("Async implementation not available")

In [15]:
# Initialize the tool
recommendation_tool = CityRecommendationTool()

# Example usage
result = recommendation_tool.run("Sam Controlman")
print(result)

Based on your preferences: 'skiing'

Here are 3 recommended cities:

1. Aspen, Colorado, USA
   Matching aspects: Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers.

2. Banff, Canada
   Matching aspects: Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers.

3. Queenstown, New Zealand
   Matching aspects: Queenstown is an adventure capital surroun

In [16]:
# # Example of inserting a user with preferences
# conn = sqlite3.connect('user_preferences.db')
# cursor = conn.cursor()

# attractions = [('Statue of Liberty & Ellis Island', 'Historical', 'Iconic landmarks of American history and immigration.', 'New York City'),
#     ('9/11 Memorial & Museum', 'Historical', 'Commemorates the victims of the 2001 attacks.', 'New York City'),
#     ('American Museum of Natural History', 'Historical', 'A treasure trove of historical exhibits.', 'New York City'),
#     ('Central Park', 'Experiences', 'A sprawling urban park for relaxation and recreation.', 'New York City'),
#     ('Broadway Theatres', 'Experiences', 'Famous for world-class theatrical productions.', 'New York City'),
#     ('The High Line', 'Experiences', 'A unique elevated park with stunning city views.', 'New York City'),
#     ('Yankee Stadium', 'Sport', 'Home to the New York Yankees baseball team.', 'New York City'),
#     ('Madison Square Garden', 'Sport', 'A multipurpose indoor arena hosting sports and concerts.', 'New York City'),
#     ('Empire State Building', 'Adventure', 'An iconic skyscraper offering panoramic views of NYC.', 'New York City'),
#     ('Rockefeller Center', 'Adventure', 'Features an observation deck and ice-skating rink.', 'New York City'),
#     ('Brooklyn Bridge', 'Adventure', 'A historic suspension bridge with stunning vistas.', 'New York City'),
#     ('Times Square', 'Experiences', 'A bustling hub of lights, theaters, and entertainment.', 'New York City'),
#     ('Maroon Bells', 'Adventure', 'Stunning mountain peaks with picturesque hiking and photography opportunities.', 'Aspen'),
#     ('Aspen Mountain', 'Sport', 'Popular for skiing and snowboarding with breathtaking views.', 'Aspen'),
#     ('Independence Pass', 'Adventure', 'A scenic mountain pass with dramatic vistas and hiking trails.', 'Aspen'),
#     ('John Denver Sanctuary', 'Experiences', 'A peaceful park with poetry-engraved rocks and serene landscapes.', 'Aspen'),
#     ('Aspen Art Museum', 'Experiences', 'Contemporary art exhibits in a sleek, modern space.', 'Aspen'),
#     ('Snowmass Village', 'Sport', 'Known for family-friendly skiing and year-round outdoor activities.', 'Aspen'),
#     ('Aspen Music Festival', 'Experiences', 'Renowned classical music performances during the summer season.', 'Aspen'),
#     ('Buttermilk Mountain', 'Sport', 'A ski resort famous for hosting the Winter X Games.', 'Aspen'),
#     ('Rio Grande Trail', 'Adventure', 'A long trail for biking, walking, and scenic exploration.', 'Aspen'),
#     ('Ashcroft Ghost Town', 'Historical', 'A preserved mining town offering a glimpse into Aspen’s history.', 'Aspen'),
#     ('Fushimi Inari Taisha', 'Historical', 'A famous Shinto shrine known for its thousands of vibrant torii gates.', 'Kyoto'),
#     ('Kinkaku-ji (Golden Pavilion)', 'Historical', 'A stunning golden Zen temple surrounded by tranquil gardens.', 'Kyoto'),
#     ('Arashiyama Bamboo Grove', 'Experiences', 'A serene and picturesque bamboo forest perfect for walking and photography.', 'Kyoto'),
#     ('Kiyomizu-dera Temple', 'Historical', 'A historic Buddhist temple with incredible views of Kyoto.', 'Kyoto'),
#     ('Nijo Castle', 'Historical', 'A former shogunate palace showcasing traditional Japanese architecture.', 'Kyoto'),
#     ('Gion District', 'Experiences', 'Famous for its traditional wooden machiya houses and geisha culture.', 'Kyoto'),
#     ('Philosopher\'s Path', 'Adventure', 'A scenic walking trail lined with cherry blossoms and small temples.', 'Kyoto'),
#     ('To-ji Temple', 'Historical', 'A UNESCO World Heritage site with a five-story pagoda.', 'Kyoto'),
#     ('Nanzen-ji Temple', 'Historical', 'A peaceful Zen temple with a stunning aqueduct and gardens.', 'Kyoto'),
#     ('Kyoto Imperial Palace', 'Historical', 'The former residence of Japan’s Imperial Family.', 'Kyoto'),
#     ('Eiffel Tower', 'Historical', 'Iconic wrought-iron lattice tower offering stunning views of Paris.', 'Paris'),
#     ('Louvre Museum', 'Historical', 'The world\'s largest art museum and a historic monument.', 'Paris'),
#     ('Notre-Dame Cathedral', 'Historical', 'A masterpiece of French Gothic architecture.', 'Paris'),
#     ('Montmartre', 'Experiences', 'Charming hilltop neighborhood with art, history, and scenic views.', 'Paris'),
#     ('Champs-Élysées', 'Experiences', 'Famous avenue lined with luxury shops and cafes.', 'Paris'),
#     ('Palace of Versailles', 'Historical', 'Lavish royal residence with stunning gardens.', 'Paris'),
#     ('Seine River Cruises', 'Adventure', 'A scenic boat ride along the Seine River.', 'Paris'),
#     ('Sacre-Coeur Basilica', 'Historical', 'Beautiful basilica atop Montmartre hill.', 'Paris'),
#     ('Musée d\'Orsay', 'Experiences', 'Art museum in a former railway station.', 'Paris'),
#     ('Arc de Triomphe', 'Historical', 'Triumphal arch honoring soldiers of the French Revolution and Napoleonic Wars.', 'Paris'),
#     ('Sydney Opera House', 'Historical', 'World-famous performing arts center with a unique design.', 'Sydney'),
#     ('Sydney Harbour Bridge', 'Adventure', 'An iconic steel arch bridge offering bridge climbs.', 'Sydney'),
#     ('Bondi Beach', 'Experiences', 'Popular beach known for surfing and vibrant atmosphere.', 'Sydney'),
#     ('Taronga Zoo', 'Experiences', 'Wildlife park with stunning views of the Sydney skyline.', 'Sydney'),
#     ('Royal Botanic Garden', 'Experiences', 'Scenic gardens near the Sydney Harbour.', 'Sydney'),
#     ('Darling Harbour', 'Experiences', 'Entertainment district with shops, restaurants, and attractions.', 'Sydney'),
#     ('Blue Mountains', 'Adventure', 'A stunning natural area with trails and breathtaking vistas.', 'Sydney'),
#     ('Manly Beach', 'Experiences', 'A beachside suburb with surfing, snorkeling, and seaside walks.', 'Sydney'),
#     ('Art Gallery of New South Wales', 'Experiences', 'A leading gallery featuring Australian and international art.', 'Sydney'),
#     ('Sea Life Sydney Aquarium', 'Experiences', 'Marine exhibits with diverse sea creatures.', 'Sydney'),
#     ('Table Mountain', 'Adventure', 'Iconic flat-topped mountain offering panoramic views.', 'Cape Town'),
#     ('Robben Island', 'Historical', 'Former prison and UNESCO World Heritage site.', 'Cape Town'),
#     ('Cape Point', 'Adventure', 'Scenic area at the tip of the Cape Peninsula.', 'Cape Town'),
#     ('Kirstenbosch National Botanical Garden', 'Experiences', 'A beautiful garden showcasing South Africa\'s flora.', 'Cape Town'),
#     ('V&A Waterfront', 'Experiences', 'Shopping and dining hub by the harbor.', 'Cape Town'),
#     ('Boulders Beach', 'Adventure', 'Famous for its colony of African penguins.', 'Cape Town'),
#     ('Bo-Kaap', 'Historical', 'Colorful neighborhood with rich Cape Malay heritage.', 'Cape Town'),
#     ('Signal Hill', 'Adventure', 'Scenic spot for sunsets and city views.', 'Cape Town'),
#     ('Castle of Good Hope', 'Historical', 'A 17th-century fort and the oldest colonial building in South Africa.', 'Cape Town'),
#     ('Chapman\'s Peak Drive', 'Adventure', 'A spectacular coastal road with incredible views.', 'Cape Town'),
#     ('Sagrada Familia', 'Historical', 'Gaudí’s unfinished masterpiece and iconic basilica.', 'Barcelona'),
#     ('Park Güell', 'Experiences', 'Colorful park with artistic designs by Gaudí.', 'Barcelona'),
#     ('La Rambla', 'Experiences', 'Vibrant pedestrian street with shops and cafes.', 'Barcelona'),
#     ('Casa Batlló', 'Historical', 'Modernist building with striking architecture.', 'Barcelona'),
#     ('Gothic Quarter', 'Historical', 'Historic neighborhood with medieval streets.', 'Barcelona'),
#     ('Camp Nou', 'Sport', 'The stadium of FC Barcelona.', 'Barcelona'),
#     ('Magic Fountain of Montjuïc', 'Experiences', 'Spectacular fountain shows with lights and music.', 'Barcelona'),
#     ('Picasso Museum', 'Experiences', 'Museum showcasing works by Pablo Picasso.', 'Barcelona'),
#     ('Barceloneta Beach', 'Adventure', 'Popular beach for relaxation and water sports.', 'Barcelona'),
#     ('Montjuïc Castle', 'Historical', 'Historic fortress offering stunning views of the city.', 'Barcelona'),
#     ('Oia Village', 'Experiences', 'Famous for whitewashed houses and stunning sunsets.', 'Santorini'),
#     ('Red Beach', 'Adventure', 'A beach with unique red volcanic sand.', 'Santorini'),
#     ('Fira Town', 'Experiences', 'The bustling capital with shops and breathtaking views.', 'Santorini'),
#     ('Akrotiri Archaeological Site', 'Historical', 'Preserved ruins of a Bronze Age settlement.', 'Santorini'),
#     ('Amoudi Bay', 'Adventure', 'Picturesque bay known for swimming and seafood.', 'Santorini'),
#     ('Santorini Volcano', 'Adventure', 'A volcanic island with hiking and hot springs.', 'Santorini'),
#     ('Perissa Black Sand Beach', 'Adventure', 'A unique beach with black volcanic sand.', 'Santorini'),
#     ('Santo Wines Winery', 'Experiences', 'Winery offering wine tastings with caldera views.', 'Santorini'),
#     ('Pyrgos Village', 'Experiences', 'A traditional village with panoramic views.', 'Santorini'),
#     ('Museum of Prehistoric Thera', 'Historical', 'Artifacts from ancient Santorini.', 'Santorini'),
#     ('Hallgrímskirkja', 'Historical', 'Iconic church with unique architecture and tower views.', 'Reykjavik'),
#     ('Blue Lagoon', 'Experiences', 'Geothermal spa with milky-blue waters.', 'Reykjavik'),
#     ('Harpa Concert Hall', 'Experiences', 'Modern glass concert hall and conference center.', 'Reykjavik'),
#     ('Sun Voyager', 'Experiences', 'A striking sculpture symbolizing exploration.', 'Reykjavik'),
#     ('Perlan', 'Experiences', 'Museum and observation deck with panoramic city views.', 'Reykjavik'),
#     ('Laugavegur', 'Experiences', 'Main shopping and dining street.', 'Reykjavik'),
#     ('Reykjavik Maritime Museum', 'Historical', 'Museum showcasing Iceland\'s maritime history.', 'Reykjavik'),
#     ('Videy Island', 'Adventure', 'Scenic island with hiking trails and public art.', 'Reykjavik'),
#     ('National Museum of Iceland', 'Historical', 'Artifacts showcasing Icelandic history and culture.', 'Reykjavik'),
#     ('Tjörnin', 'Experiences', 'A scenic city pond surrounded by colorful houses.', 'Reykjavik'),
#     ('Tokyo Tower', 'Historical', 'Iconic tower inspired by the Eiffel Tower.', 'Tokyo'),
#     ('Meiji Shrine', 'Historical', 'Shinto shrine surrounded by a peaceful forest.', 'Tokyo'),
#     ('Senso-ji Temple', 'Historical', 'Tokyo\'s oldest Buddhist temple.', 'Tokyo'),
#     ('Shibuya Crossing', 'Experiences', 'Famous bustling intersection in Tokyo.', 'Tokyo'),
#     ('Akihabara', 'Experiences', 'Hub of electronics, anime, and otaku culture.', 'Tokyo'),
#     ('Tsukiji Outer Market', 'Experiences', 'Famous market offering fresh seafood and local delicacies.', 'Tokyo'),
#     ('Tokyo Disneyland & DisneySea', 'Experiences', 'World-class theme parks.', 'Tokyo'),
#     ('Odaiba', 'Experiences', 'Man-made island with shopping and entertainment.', 'Tokyo'),
#     ('Ueno Park', 'Experiences', 'Large park with museums, a zoo, and cherry blossoms.', 'Tokyo'),
#     ('Ginza', 'Experiences', 'Upscale shopping district with luxury brands.', 'Tokyo'),
#     ('Colosseum', 'Historical', 'Iconic ancient amphitheater used for gladiatorial contests.', 'Rome'),
#     ('Roman Forum', 'Historical', 'Ruins of ancient government buildings and temples.', 'Rome'),
#     ('Vatican Museums', 'Historical', 'Renowned museums housing masterpieces like the Sistine Chapel.', 'Rome'),
#     ('St. Peter\'s Basilica', 'Historical', 'Majestic basilica in Vatican City.', 'Rome'),
#     ('Pantheon', 'Historical', 'Well-preserved ancient Roman temple.', 'Rome'),
#     ('Trevi Fountain', 'Experiences', 'Famous Baroque fountain known for coin tossing.', 'Rome'),
#     ('Piazza Navona', 'Experiences', 'Vibrant square with fountains, cafes, and street performers.', 'Rome'),
#     ('Villa Borghese Gardens', 'Experiences', 'Beautiful landscaped gardens with museums and a lake.', 'Rome'),
#     ('Trastevere', 'Experiences', 'Charming neighborhood with cobbled streets and vibrant nightlife.', 'Rome'),
#     ('Castel Sant\'Angelo', 'Historical', 'Historic fortress and papal residence.', 'Rome'),
#     ('Banff National Park', 'Adventure', 'Canada\'s first national park with mountains, forests, and lakes.', 'Banff'),
#     ('Lake Louise', 'Adventure', 'Glacial lake known for its turquoise waters.', 'Banff'),
#     ('Moraine Lake', 'Adventure', 'Stunning glacial-fed lake in the Valley of the Ten Peaks.', 'Banff'),
#     ('Banff Gondola', 'Adventure', 'Cable car ride offering panoramic views of the Rockies.', 'Banff'),
#     ('Sulphur Mountain', 'Adventure', 'Hiking trails and hot springs with incredible views.', 'Banff'),
#     ('Johnston Canyon', 'Adventure', 'Scenic canyon with waterfalls and hiking paths.', 'Banff'),
#     ('Bow Falls', 'Adventure', 'Beautiful waterfall near the town of Banff.', 'Banff'),
#     ('Banff Upper Hot Springs', 'Experiences', 'Relaxing thermal hot springs.', 'Banff'),
#     ('Tunnel Mountain', 'Adventure', 'Popular hiking trail with panoramic views.', 'Banff'),
#     ('Whyte Museum of the Canadian Rockies', 'Historical', 'Museum showcasing the culture and history of the Rockies.', 'Banff'),
#     ('La Boca', 'Experiences', 'Colorful neighborhood known for tango and street art.', 'Buenos Aires'),
#     ('Teatro Colón', 'Historical', 'Iconic opera house with stunning architecture.', 'Buenos Aires'),
#     ('Recoleta Cemetery', 'Historical', 'Resting place of notable figures like Eva Perón.', 'Buenos Aires'),
#     ('Casa Rosada', 'Historical', 'Presidential palace with a rich political history.', 'Buenos Aires'),
#     ('San Telmo Market', 'Experiences', 'Bustling market with antiques and local crafts.', 'Buenos Aires'),
#     ('Plaza de Mayo', 'Historical', 'Historic square surrounded by iconic buildings.', 'Buenos Aires'),
#     ('Palermo Soho', 'Experiences', 'Trendy area with boutiques, cafes, and nightlife.', 'Buenos Aires'),
#     ('El Ateneo Grand Splendid', 'Experiences', 'A historic theater converted into a stunning bookstore.', 'Buenos Aires'),
#     ('Reserva Ecológica Costanera Sur', 'Adventure', 'Large urban nature reserve with trails and wildlife.', 'Buenos Aires'),
#     ('Museum of Latin American Art of Buenos Aires (MALBA)', 'Experiences', 'Showcasing Latin American modern art.', 'Buenos Aires'),
#     ('Burj Khalifa', 'Historical', 'The world\'s tallest building with observation decks.', 'Dubai'),
#     ('Palm Jumeirah', 'Adventure', 'Man-made island shaped like a palm tree.', 'Dubai'),
#     ('Dubai Mall', 'Experiences', 'One of the largest malls with shops, restaurants, and an aquarium.', 'Dubai'),
#     ('Desert Safari', 'Adventure', 'Exciting dune bashing and cultural experiences in the desert.', 'Dubai'),
#     ('Burj Al Arab', 'Historical', 'Iconic luxury hotel shaped like a sail.', 'Dubai'),
#     ('Dubai Marina', 'Experiences', 'A modern waterfront area with dining and entertainment.', 'Dubai'),
#     ('Global Village', 'Experiences', 'Cultural and entertainment hub showcasing pavilions from around the world.', 'Dubai'),
#     ('Dubai Fountain', 'Experiences', 'Spectacular choreographed water shows.', 'Dubai'),
#     ('Ski Dubai', 'Adventure', 'Indoor ski resort with real snow.', 'Dubai'),
#     ('Al Fahidi Historical District', 'Historical', 'Historic area showcasing Dubai\'s traditional architecture.', 'Dubai'),
#     ('Lake Wakatipu', 'Adventure', 'Stunning lake surrounded by mountains.', 'Queenstown'),
#     ('Skyline Queenstown', 'Adventure', 'Cable car ride with scenic views and adventure activities.', 'Queenstown'),
#     ('Milford Sound', 'Adventure', 'Spectacular fjord with waterfalls and wildlife.', 'Queenstown'),
#     ('The Remarkables', 'Sport', 'Renowned ski area with breathtaking views.', 'Queenstown'),
#     ('Shotover Jet', 'Adventure', 'High-speed jet boating experience.', 'Queenstown'),
#     ('Queenstown Gardens', 'Experiences', 'Beautiful botanical gardens by the lake.', 'Queenstown'),
#     ('Bungy Jumping at Kawarau Bridge', 'Adventure', 'Home to the world\'s first commercial bungy jump.', 'Queenstown'),
#     ('Arrowtown', 'Historical', 'Charming historic town from the gold rush era.', 'Queenstown'),
#     ('Glenorchy', 'Adventure', 'Scenic area near Queenstown, perfect for hiking.', 'Queenstown'),
#     ('TSS Earnslaw Steamship Cruise', 'Experiences', 'Historic steamship cruises on Lake Wakatipu.', 'Queenstown')]

# cursor.executemany('''
#     INSERT INTO attractions (name, category, description, city) 
#     VALUES (?, ?, ?, ?)
# ''', attractions)

# conn.commit()
# conn.close()

In [17]:
import requests
import time
from typing import Dict, List, Optional, Tuple
from pydantic import Field

class CityAttractionsTool(BaseTool):
    name: str = "city_attractions_tool"
    description: str = """
    Fetches attractions for a given city from the database and retrieves their coordinates.
    Input should be a city name (string).
    Returns a list of attractions with their details and coordinates.
    """
    conn: sqlite3.Connection = Field(
        default_factory=lambda: sqlite3.connect('user_preferences.db', check_same_thread=False),
        exclude=True
    )
    cursor: sqlite3.Cursor = Field(
        default=None,
        exclude=True
    )
    
    def __init__(self, **data):
        super().__init__(**data)
        if self.cursor is None:
            self.cursor = self.conn.cursor()
    
    def get_coordinates(self, location: str, city: str) -> Optional[Tuple[float, float]]:
        """Get coordinates for a location using Nominatim geocoding service"""
        try:
            # Add city name to make the search more accurate
            search_query = f"{location}, {city}"
            
            # Use Nominatim API with a custom User-Agent
            url = "https://nominatim.openstreetmap.org/search"
            headers = {"User-Agent": "TravelPlannerAgent/1.0"}
            params = {
                "q": search_query,
                "format": "json",
                "limit": 1
            }
            
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            
            results = response.json()
            if results:
                lat = float(results[0]["lat"])
                lon = float(results[0]["lon"])
                return (lat, lon)
            
            return None
            
        except Exception as e:
            print(f"Error getting coordinates for {location}: {str(e)}")
            return None
        finally:
            # Be nice to the free API by adding a small delay
            time.sleep(1)
    
    def get_attractions(self, city: str) -> List[Dict]:
        """Get all attractions for a city with their coordinates"""
        try:
            self.cursor.execute('''
                SELECT name, category, description 
                FROM attractions 
                WHERE city = ?
            ''', (city,))
            
            attractions = []
            for name, category, description in self.cursor.fetchall():
                coordinates = self.get_coordinates(name, city)
                attractions.append({
                    "name": name,
                    "category": category,
                    "description": description,
                    "coordinates": coordinates
                })
            
            return attractions
            
        except Exception as e:
            print(f"Error fetching attractions: {str(e)}")
            return []
    
    def _run(self, city: str) -> str:
        """Get attractions with coordinates for a specific city"""
        attractions = self.get_attractions(city)
        
        if not attractions:
            return f"No attractions found for {city}"
        
        # Format response
        response = [f"Attractions in {city}:\n"]
        
        for attraction in attractions:
            response.append(f"- {attraction['name']}")
            response.append(f"  Category: {attraction['category']}")
            response.append(f"  Description: {attraction['description']}")
            if attraction['coordinates']:
                lat, lon = attraction['coordinates']
                response.append(f"  Coordinates: {lat:.4f}° N, {lon:.4f}° E")
            response.append("")  # Empty line for readability
        
        return "\n".join(response)
    
    def _arun(self, city: str) -> str:
        """Async version - Not implemented"""
        raise NotImplementedError("Async implementation not available")
    
    def __del__(self):
        """Clean up database connection"""
        if hasattr(self, 'conn'):
            self.conn.close()

In [18]:
# Initialize the tool
attractions_tool = CityAttractionsTool()

# Get attractions with coordinates for a city
result = attractions_tool.run("Paris")
print(result)

Attractions in Paris:

- Eiffel Tower
  Category: Historical
  Description: Iconic wrought-iron lattice tower offering stunning views of Paris.
  Coordinates: 48.8583° N, 2.2945° E

- Louvre Museum
  Category: Historical
  Description: The world's largest art museum and a historic monument.
  Coordinates: 48.8611° N, 2.3380° E

- Notre-Dame Cathedral
  Category: Historical
  Description: A masterpiece of French Gothic architecture.
  Coordinates: 48.8529° N, 2.3501° E

- Montmartre
  Category: Experiences
  Description: Charming hilltop neighborhood with art, history, and scenic views.
  Coordinates: 48.8855° N, 2.3392° E

- Champs-Élysées
  Category: Experiences
  Description: Famous avenue lined with luxury shops and cafes.
  Coordinates: 48.8708° N, 2.3053° E

- Palace of Versailles
  Category: Historical
  Description: Lavish royal residence with stunning gardens.

- Seine River Cruises
  Category: Adventure
  Description: A scenic boat ride along the Seine River.

- Sacre-Coeur Ba

In [19]:
from typing import Optional
import ast


class UserPreferencesTool(BaseTool):
    name: str = "user_preferences_tool"
    description: str = "Manages user preferences for travel recommendations"
    conn: sqlite3.Connection = Field(default_factory=lambda: sqlite3.connect('user_preferences.db', check_same_thread=False), exclude=True)
    cursor: sqlite3.Cursor = Field(default=None, exclude=True)
    
    def __init__(self, **data):
        super().__init__(**data)
        if self.cursor is None:
            self.cursor = self.conn.cursor()
    
    def _get_user_preferences(self, username: str) -> Optional[str]:
        """Check if user exists and return their preferences"""
        self.cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    
    def _store_preferences(self, username: str, preferences: str, hometown: str = None) -> None:
        """Store or update user preferences"""
        self.cursor.execute('''
            INSERT INTO users (username, preferences) 
            VALUES (?, ?)
            ON CONFLICT(username) 
            DO UPDATE SET preferences = excluded.preferences
        ''', (username, preferences))
        self.conn.commit()
    
    def _run(self, username: str, preferences: str = None, hometown: str = None) -> str:
        """
        Check if user has preferences, if not prompt for them.
        If preferences are provided, store them.
        Returns the preferences for use with recommendations.
        """
        try:
            # If preferences are provided, store them
            if preferences:
                self._store_preferences(username, preferences, hometown)
                return f"Preferences stored successfully for {username}: {preferences}"
            
            # Otherwise, check for existing preferences
            existing_preferences = self._get_user_preferences(username)
            if existing_preferences:
                return f"Found existing preferences: {existing_preferences}"
            
            # If no preferences found, this message will be sent to the agent
            return "Please tell me about your travel preferences. Consider including:\n" + \
                   "- Activities you enjoy\n" + \
                   "- Climate preferences\n" + \
                   "- Cultural interests\n" + \
                   "- Urban vs. rural settings"
        except Exception as e:
            return f"Error accessing preferences: {str(e)}"

    def update_preferences(self, username: str, new_preferences: str) -> str:
        """Store new user preferences and return confirmation"""
        try:
            self._store_preferences(username, new_preferences)
            return f"Preferences stored successfully for {username}"
        except Exception as e:
            return f"Error storing preferences: {str(e)}"
    
    def __del__(self):
        """Ensure database connection is closed"""
        if hasattr(self, 'conn'):
            self.conn.close()

In [20]:
preferences_tool = UserPreferencesTool()

username = "Sam Controlman"

# Get user preferences
preferences = preferences_tool._get_user_preferences(username)
if preferences:
    print(f"Existing preferences for {username}: {preferences}")
else:
    print(f"No preferences found for {username}.")

Existing preferences for Sam Controlman: skiing


In [21]:
# !pip install google-search-results

In [22]:
import os
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [23]:
print(os.getenv('SERPAPI_API_KEY'))

680afdfe2534e36965be2913ec4843454ef81b50795335fdbc2d059f99b8d821


In [24]:
serpapi_api_key = '680afdfe2534e36965be2913ec4843454ef81b50795335fdbc2d059f99b8d821'

In [25]:
from serpapi.google_search import GoogleSearch

params = {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "departure_id": "CDG",
    "arrival_id": "AUS",
    "outbound_date": "2024-12-02",
    "return_date": "2024-12-08",
    "currency": "USD",
    "api_key": serpapi_api_key
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'error': '`outbound_date` cannot be in the past.'}

In [26]:
params = {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "departure_id": "CDG",
    "arrival_id": "AUS",
    "outbound_date": "2024-12-02",
    "return_date": "2024-12-08",
    "currency": "USD",
    "api_key": serpapi_api_key
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'error': '`outbound_date` cannot be in the past.'}

In [27]:
from typing import Optional, Any, Dict, List, Tuple
from datetime import datetime
from pydantic import Field, BaseModel

# Define the input schema
class FlightSearchInput(BaseModel):
    departure_airport: str = Field(..., description="IATA code of departure airport (e.g., 'JFK')")
    arrival_airport: str = Field(..., description="IATA code of arrival airport (e.g., 'LAX')")
    departure_date: str = Field(..., description="Departure date in YYYY-MM-DD format")
    return_date: str = Field(None, description="Optional return date in YYYY-MM-DD format")

class FlightSearchTool(BaseTool):
    name: str = "flight_search_tool"
    description: str = """
    Searches for flight information between airports. 
    Input should be a dictionary with:
    - departure_airport: IATA code of departure airport (e.g., 'JFK')
    - arrival_airport: IATA code of arrival airport (e.g., 'LAX')
    - departure_date: Date in YYYY-MM-DD format
    """
    args_schema: type[BaseModel] = FlightSearchInput
    api_key: str = Field(..., description="SerpAPI API key")
    
    def _run(self, departure_airport: str, arrival_airport: str, 
             departure_date: str, return_date: str = None) -> str:
        """Execute the flight search"""
        try:
            type_num = 2
            if return_date is not None:
                type_num = 1
            search_params = {
                "engine": "google_flights",
                "hl": "en",
                "gl": "us",
                "departure_id": departure_airport,
                "arrival_id": arrival_airport,
                "outbound_date": departure_date,
                "type": type_num,
                "currency": "USD",
                "api_key": self.api_key
            }
            
            if return_date:
                search_params["return_date"] = return_date
            
            # Rest of your existing search code...
            search = GoogleSearch(search_params)
            results = search.get_dict()
            
            # Format the response
            if 'error' in results:
                return f"Error searching flights: {results['error']}"
            
            # Extract relevant flight information
            best_flights = results.get('best_flights', [])
            if not best_flights:
                return "No flights found for the specified criteria."
            
            # Format response
            response = "Found the following flights:\n"
            for flight in best_flights[:3]:  # Show top 3 results
                price = flight.get('price', 'N/A')
                duration = flight.get('total_duration', 'N/A')
                airline = flight['flights'][0].get('airline', 'N/A')
                departure_time = flight['flights'][0]['departure_airport'].get('time', 'N/A')
                arrival_time = flight['flights'][0]['arrival_airport'].get('time', 'N/A')
                
                response += f"- {airline}: ${price}, Duration: {duration}min ({departure_time} - {arrival_time})\n"
            
            return response
            
        except Exception as e:
            return f"Error processing flight search: {str(e)}"

    def _arun(self, query: str):
        raise NotImplementedError("Async implementation not available")
    
  
    def _validate_date(self, date_str: str) -> bool:
        """Validate date format and ensure it's not in the past"""
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            return date >= datetime.now()
        except ValueError:
            return False

In [28]:
flight_tool = FlightSearchTool(api_key=serpapi_api_key)

# Example input
test_input = {
    'departure_airport': 'JFK',
    'arrival_airport': 'LAX',
    'departure_date': '2024-12-15',
    'return_date': '2024-12-20'
}

result = flight_tool.run(test_input)
print(result)

Found the following flights:
- Delta: $897, Duration: 381min (2024-12-15 09:55 - 2024-12-15 13:16)
- American: $907, Duration: 385min (2024-12-15 08:00 - 2024-12-15 11:25)
- JetBlue: $952, Duration: 380min (2024-12-15 09:00 - 2024-12-15 12:20)



In [29]:
params = {
  "api_key": serpapi_api_key,
  "engine": "google_hotels",
  "q": "Bali Resorts",
  "hl": "en",
  "gl": "us",
  "check_in_date": "2024-12-03",
  "check_out_date": "2024-12-04",
  "currency": "USD"
}

search = GoogleSearch(params)
results = search.get_dict()

In [30]:
class HotelSearchInput(BaseModel):
    destination: str = Field(..., description="Name of destination city/area (e.g., 'Bali Resorts')")
    check_in_date: str = Field(..., description="Check-in date in YYYY-MM-DD format")
    check_out_date: str = Field(..., description="Check-out date in YYYY-MM-DD format")
    
class HotelSearchTool(BaseTool):
    name: str = "hotel_search_tool"
    description: str = """
    Searches for hotels at a given destination.
    Input should be a dictionary with:
    - destination: Name of destination city/area (e.g., 'Bali Resorts')
    - check_in_date: Date in YYYY-MM-DD format
    - check_out_date: Date in YYYY-MM-DD format
    """
    args_schema: type[BaseModel] = HotelSearchInput
    api_key: str = Field(..., description="SerpAPI API key")
    
    def _run(self, destination: str, check_in_date: str, check_out_date: str) -> str:
        """Execute the hotel search"""
        try:
            # Validate dates
            if not (self._validate_date(check_in_date) and self._validate_date(check_out_date)):
                return "Error: Please provide valid future dates in YYYY-MM-DD format."
            
            search_params = {
                "api_key": self.api_key,
                "engine": "google_hotels",
                "q": destination,
                "hl": "en",
                "gl": "us",
                "check_in_date": check_in_date,
                "check_out_date": check_out_date,
                "currency": "USD"
            }
            
            search = GoogleSearch(search_params)
            results = search.get_dict()
            
            # Format the response
            if 'error' in results:
                return f"Error searching hotels: {results['error']}"
            
            # Extract hotel information
            hotels = results.get('properties', [])
            if not hotels:
                return f"No hotels found in {destination} for the specified dates."
            
            # Format response
            response = f"Found the following hotels in {destination}:\n\n"
            for hotel in hotels[:5]:  # Show top 5 results
                name = hotel.get('name', 'N/A')
                price_per_night_lowest = hotel.get('rate_per_night', {}).get('lowest', 'N/A')
                total_price_lowest = hotel.get('total_rate', {}).get('lowest', 'N/A')
                rating = hotel.get('overall_rating', 'N/A')
                reviews = hotel.get('reviews', 'N/A')
                description = hotel.get('essential_info', 'No description available')
                latitude = hotel.get('gps_coordinates', {}).get('latitude', 'N/A')
                longitude = hotel.get('gps_coordinates', {}).get('longitude', 'N/A')
                
                response += f"🏨 {name}\n"
                response += f"   Price: {total_price_lowest} total for your stay\n"
                response += f"   Rating: {rating}/5 ({reviews} reviews)\n"
                response += f"   Description: {description[:200]}...\n\n"
                response += f"   Address: {str(latitude) + ", " + str(longitude)}\n\n"
            
            return response
            
        except Exception as e:
            return f"Error processing hotel search: {str(e)}"

    def _arun(self, query: str):
        raise NotImplementedError("Async implementation not available")
    
    def _validate_date(self, date_str: str) -> bool:
        """Validate date format and ensure it's not in the past"""
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            return date >= datetime.now()
        except ValueError:
            return False

In [31]:
hotel_tool = HotelSearchTool(api_key=serpapi_api_key)

# Example usage
test_input = {
    'destination': 'Bali',
    'check_in_date': '2024-12-15',
    'check_out_date': '2024-12-20'
}

result = hotel_tool.run(test_input)
print(result)

Found the following hotels in Bali:

🏨 Sea Bridge Villa Ceningan
   Price: $191 total for your stay
   Rating: 4.8/5 (125 reviews)
   Description: ['Entire house', 'Sleeps 5', '2 bedrooms', '1 bathroom', '3 beds', '43 sq ft']...

   Address: -8.695569038391113, 115.45111083984375

🏨 Bali Bohemia Huts - Double Room with Balcony
   Price: $312 total for your stay
   Rating: 4.7/5 (666 reviews)
   Description: ['Entire villa', 'Sleeps 2', '1 bedroom', '1 bathroom', '1 bed']...

   Address: -8.519630432128906, 115.25818634033203

🏨 Kumpul Kumpul Villa Seminyak
   Price: $281 total for your stay
   Rating: 4.403774/5 (159 reviews)
   Description: ['Entire villa', 'Sleeps 2', '3 bedrooms', '3 bathrooms', '1 bed']...

   Address: -8.695887565612793, 115.16898345947266

🏨 Aleesha Villas
   Price: $232 total for your stay
   Rating: 4.174606/5 (469 reviews)
   Description: ['Entire villa', 'Sleeps 2', '1 bedroom', '6 bathrooms', '1 bed']...

   Address: -8.697257041931152, 115.25972747802734

🏨

In [32]:
class CalculatorTool(BaseTool):
    name: str = "calculator_tool"
    description: str = """
    Performs basic mathematical calculations. Can handle:
    - Addition (+)
    - Subtraction (-)
    - Multiplication (*)
    - Division (/)
    - Exponents (**)
    - Parentheses ()
    
    Input should be a mathematical expression as a string (e.g., "2 + 2" or "(23 * 4.5) / 2").
    """
    
    def _run(self, expression: str) -> str:
        """Execute the calculation"""
        try:
            # Remove any dangerous functions/attributes
            if any(x in expression.lower() for x in ['import', 'eval', 'exec', 'getattr', '__']):
                return "Error: Invalid expression. Please use only basic mathematical operations."
            
            # Calculate the result
            result = eval(expression, {"__builtins__": {}})
            
            # Format the result
            if isinstance(result, (int, float)):
                # Handle integer results
                if result.is_integer():
                    return f"{int(result)}"
                # Format float results to 4 decimal places
                return f"{result:.4f}".rstrip('0').rstrip('.')
            
            return str(result)
            
        except ZeroDivisionError:
            return "Error: Division by zero"
        except Exception as e:
            return f"Error calculating result: {str(e)}"
    
    def _arun(self, expression: str):
        """Async implementation not available"""
        raise NotImplementedError("Async implementation not available")

In [33]:
def plot_multiple_coordinates(coordinates):
    # Create a map centered on the first coordinate
    center_lat = coordinates[0][0]
    center_lon = coordinates[0][1]
    map_location = folium.Map(location=[center_lat, center_lon], zoom_start=10)
    
    # Add markers for each coordinate
    for lat, lon, name in coordinates:
        folium.Marker(
            [lat, lon],
            popup=name,
            tooltip=f'Coordinates: {lat}, {lon}'
        ).add_to(map_location)
    
    # Save the map
    map_location.save('multiple_locations_map.html')
    print(f"Map created with {len(coordinates)} locations")

# Example usage
coordinates = [
    (41.397158, 2.160873, 'Barcelona Location'),
    (40.4168, -3.7038, 'Madrid Location'),
    (48.8566, 2.3522, 'Paris Location')
]
plot_multiple_coordinates(coordinates)

Map created with 3 locations


In [34]:
from pydantic import BaseModel, Field

class MapCoordinates(BaseModel):
    coordinates: List[Tuple[float, float, str]] = Field(
        ..., 
        description="List of tuples containing (latitude, longitude, location_name)"
    )
    filename: str = Field(
        default="travel_map.html",
        description="Name of the output HTML file"
    )

class MapPlottingTool(BaseTool):
    name: str = "map_plotting_tool"
    description: str = """
    Creates an interactive map with markers for specified locations.
    Input should be a dictionary with:
    - coordinates: List of tuples, each containing (latitude, longitude, location_name)
    - filename: Name for the output HTML file, eg. username+destination+dates.html
    
    Example input:
    {
        'coordinates': [(41.397158, 2.160873, 'Barcelona'), (48.8566, 2.3522, 'Paris')],
        'filename': 'jason_derulo_paris_12102024.html'
    }
    """
    args_schema: type[BaseModel] = MapCoordinates

    def _run(self, coordinates: List[Tuple[float, float, str]], filename: str = "travel_map.html") -> str:
        """Create a map with the given coordinates"""
        try:
            if not coordinates:
                return "Error: No coordinates provided"

            # Create a map centered on the first coordinate
            center_lat = coordinates[0][0]
            center_lon = coordinates[0][1]
            map_location = folium.Map(
                location=[center_lat, center_lon],
                zoom_start=4,  # Adjusted zoom to show multiple locations better
                tiles='OpenStreetMap'
            )

            # Add markers for each coordinate
            for lat, lon, name in coordinates:
                folium.Marker(
                    [lat, lon],
                    popup=name,
                    tooltip=f'{name}\nCoordinates: {lat:.4f}, {lon:.4f}'
                ).add_to(map_location)

            # Add lines connecting the locations in order
            if len(coordinates) > 1:
                points = [[lat, lon] for lat, lon, _ in coordinates]
                folium.PolyLine(
                    points,
                    weight=2,
                    color='red',
                    opacity=0.8
                ).add_to(map_location)

            # Save the map
            map_location.save(filename)
            return f"Map successfully created with {len(coordinates)} locations and saved as '{filename}'"

        except Exception as e:
            return f"Error creating map: {str(e)}"

    def _arun(self, coordinates: List[Tuple[float, float, str]], filename: str = "travel_map.html"):
        """Async implementation not available"""
        raise NotImplementedError("Async implementation not available")

In [35]:
# Initialize the tool
map_tool = MapPlottingTool()

# Example usage
test_coordinates = [
    (41.397158, 2.160873, 'Barcelona'),
    (40.4168, -3.7038, 'Madrid'),
    (48.8566, 2.3522, 'Paris')
]

result = map_tool.run({
    'coordinates': test_coordinates,
    'filename': 'test_trip_map.html'
})
print(result)

Map successfully created with 3 locations and saved as 'test_trip_map.html'


In [36]:
from langchain.tools import BaseTool
from pydantic import Field, BaseModel
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import List, Dict, Optional
from datetime import datetime
import json
import os

class GmailItineraryInput(BaseModel):
    """Input schema for the Gmail Itinerary Tool"""
    to_email: str
    username: str
    flight_details: List[Dict]
    hotel_details: Dict
    attractions: List[Dict]
    map_url: Optional[str] = None

class GmailItineraryTool(BaseTool):
    name: str = "gmail_itinerary_tool"
    description: str = """
    Sends an itinerary email including flights, hotel, points of attraction, and a map using Gmail.
    The input should be a JSON string containing:
    {
        "to_email": "recipient's email",
        "username": "traveler's name",
        "flight_details": [list of flight information],
        "hotel_details": {hotel booking information},
        "attractions": [list of attractions],
        "map_url": "optional map URL"
    }
    """
    
    smtp_server: str = Field(default="smtp.gmail.com")
    smtp_port: int = Field(default=587)
    smtp_user: str = Field(default="your.email@gmail.com")
    smtp_password: str = Field(default="your-app-password")

    def format_itinerary_email(self, data: GmailItineraryInput) -> str:
        """Format the email body with itinerary details"""
        
        email_body = f"""
Dear {data.username},

Thank you for booking your trip! Here's your complete itinerary:

FLIGHT DETAILS:
"""
        # Add flight details
        for flight in data.flight_details:
            email_body += f"""
🛫 {flight.get('airline', 'Airline')} - ${flight.get('price', 'N/A')}
   Departure: {flight.get('departure_time', 'N/A')}
   Arrival: {flight.get('arrival_time', 'N/A')}
   Duration: {flight.get('duration', 'N/A')} minutes
"""

        # Add hotel details
        email_body += f"""
HOTEL DETAILS:
🏨 {data.hotel_details.get('name', 'Hotel Name')}
   Address: {data.hotel_details.get('address', 'N/A')}
   Check-in: {data.hotel_details.get('check_in', 'N/A')}
   Check-out: {data.hotel_details.get('check_out', 'N/A')}
   Room Type: {data.hotel_details.get('room_type', 'N/A')}
"""

        # Add attractions
        email_body += """
POINTS OF INTEREST:
"""
        for attraction in data.attractions:
            email_body += f"""
🎯 {attraction.get('name', 'Attraction Name')}
   Category: {attraction.get('category', 'N/A')}
   Description: {attraction.get('description', 'N/A')}
"""

        if data.map_url:
            email_body += f"""
MAP:
🗺️ View your trip map here: {data.map_url}
"""

        email_body += """

Have a great trip!
Best regards,
Fred, Your Travel Assistant
"""
        return email_body

    def _run(self, tool_input: str) -> str:
        """Execute the email sending tool"""
        try:
            # Parse the input JSON string into a dictionary
            if isinstance(tool_input, str):
                input_data = json.loads(tool_input)
            else:
                input_data = tool_input
                
            # Convert to GmailItineraryInput
            data = GmailItineraryInput(**input_data)

            # Create the email
            msg = MIMEMultipart()
            msg['From'] = self.smtp_user
            msg['To'] = data.to_email
            msg['Subject'] = f"Your Travel Itinerary - {datetime.now().strftime('%B %d, %Y')}"

            # Format and attach the body text
            body = self.format_itinerary_email(data)
            msg.attach(MIMEText(body, 'plain'))

            # Send the email
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(self.smtp_user, self.smtp_password)
                server.send_message(msg)

            return f"Itinerary email sent successfully to {data.to_email}"

        except Exception as e:
            return f"Error sending email: {str(e)}"

    def _arun(self, tool_input: str) -> str:
        raise NotImplementedError("Async version not implemented")

# Example usage
def create_gmail_tool(gmail_address: str, app_password: str) -> GmailItineraryTool:
    return GmailItineraryTool(
        smtp_user=gmail_address,
        smtp_password=app_password
    )


In [37]:
gmail_tool = create_gmail_tool(
        gmail_address="h.belagur1980@gmail.com",
        app_password=os.getenv('GMAIL_PASSWORD')
    )

 # Example data
sample_input = {
        "to_email": "hb2779@columbia.edu",
        "username": "John Doe",
        "flight_details": [
            {
                "airline": "American Airlines",
                "price": 254,
                "departure_time": "2024-03-15 08:00",
                "arrival_time": "2024-03-15 09:29",
                "duration": 391
            }
        ],
        "hotel_details": {
            "name": "Sample Hotel",
            "address": "123 Main St, City, State",
            "check_in": "2024-03-15",
            "check_out": "2024-03-20",
            "room_type": "Deluxe Double"
        },
        "attractions": [
            {
                "name": "Sample Attraction",
                "category": "Historical",
                "description": "A beautiful historic site"
            }
        ],
        "map_url": "https://example.com/map"
    }

    # Convert to JSON string
# tool_input = json.dumps(sample_input)

    # Send the email
# result = gmail_tool.run(tool_input)
# print(result)

In [38]:
from typing import TypedDict, Annotated, Sequence, Union
from langgraph.graph import Graph, StateGraph
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

In [39]:
preferences_tool = UserPreferencesTool()
flight_tool = FlightSearchTool(api_key=serpapi_api_key)
destination_recommendation_tool = CityRecommendationTool()
hotel_tool = HotelSearchTool(api_key=serpapi_api_key)
calculator_tool = CalculatorTool()
map_tool = MapPlottingTool()
attractions_tool = CityAttractionsTool()
email_tool = create_gmail_tool(
        gmail_address="h.belagur1980@gmail.com",
        app_password=os.getenv('GMAIL_PASSWORD')
)


tools = [preferences_tool, flight_tool, destination_recommendation_tool, hotel_tool, calculator_tool, attractions_tool, email_tool]

system_prompt = f"""You are a helpful travel assistant named Fred. Follow these steps in order:

1. INITIAL GREETING:
   - Get the user's name
   - Using the name, check if they have stored preferences using the user_preferences_tool
   
2. PREFERENCES (only if needed):
   - If no preferences exist, ask about their travel preferences
   - Store their preferences using user_preferences_tool
   
3. CITY RECOMMENDATION:
   - Use city_recommendation_tool EXACTLY ONCE to suggest cities
   - Ask the user if they'd like to search for flights to any of the recommended cities
   
4. FLIGHT SEARCH (when user selects a destination):
   - Use the user's hometown as the departure_airport, if it is not known, get it from the user
   - Ask for specific travel dates (departure and return, if return is not given, get the duration of the trip)
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}

5. NEXT STEPS:
   - After showing flight results, ask if they want to:
     a) Search flights for different dates
     b) Search for a different destination

5. ATTRACTIONS:
   - Once the user selects a destination and a flight, use the city_attractions_tool to get the top attractions in the destination

6. MAP:
   - Using the attractions retrieved, use the map_plotting_tool to create a map of the trip
   
7. HOTEL SEARCH (when user selects a destination and a flight):
   - Look for hotels in the destination for the dates of the flight (departure and return date)

8. CALCULATOR:
   - If the user asks about the cost of a trip, use the calculator_tool to sum up the selected flight and hotel costs and also provide the breakdown of the costs   

9. END:
   - Once the user is done with the conversation, thank them, get their email, do not assume it and end the conversation telling them that you will email them the detials
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.

For eg. If the user's name is John Lincoln and they are planning a trip to New York. First, check if John Lincoln has stored preferences using the user_preferences_tool. If not, ask about his travel preferences.
Then using their preferences, recommend a city using the city_recommendation_tool. Ask the user where they are flying from and the dates of travel. Use the flight_search_tool to get the flight details.
Then look for hotels in the destination for the dates of the flight (departure and return date) using the hotel_search_tool. Then look for attractions in the destination using the city_attractions_tool.
Then create a map of the trip using the map_plotting_tool. Then calculate the cost of the trip using the calculator_tool. Then send an email to the user with the itinerary details using the gmail_itinerary_tool 
after the user is done with the conversation asking them for their email. Make sure you execute all the steps.
"""

In [40]:
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
graph = create_react_agent(model, tools=tools, state_modifier=system_prompt)


You:  Sure



Fred: ================================ Human Message =================================

Sure
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_bW4PcOmU9xGHU64tMXc9u55S)
 Call ID: call_bW4PcOmU9xGHU64tMXc9u55S
  Args:
    username: Sam Controlman
    preferences: Skiing
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Sam Controlman: Skiing
================================== Ai Message ==================================

Great, Sam Controlman! I see that you enjoy skiing and your hometown is New York. Based on your preference for skiing, I recommend visiting Aspen, Colorado. It's a fantastic destination for skiing with beautiful slopes and a charming town to explore.

Now, I'll search for flights from New York to Aspen. Let's find the best options for you.
Tool Calls:
  flight_search_tool (call_B


You:  15th december, 2024



Fred: ================================ Human Message =================================

15th december, 2024
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_r7RBCQaYC3PMzIDy6LwQLo0y)
 Call ID: call_r7RBCQaYC3PMzIDy6LwQLo0y
  Args:
    username: Sam Controlman
    preferences: Skiing
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Sam Controlman: Skiing
================================== Ai Message ==================================
Tool Calls:
  flight_search_tool (call_SjpplpR8NxMNK56mIhrLOYaY)
 Call ID: call_SjpplpR8NxMNK56mIhrLOYaY
  Args:
    departure_airport: JFK
    arrival_airport: MUC
    departure_date: 2024-12-15
  flight_search_tool (call_ERCp5NFOG0YX6LEty9NGqdla)
 Call ID: call_ERCp5NFOG0YX6LEty9NGqdla
  Args:
    departure_airport: MUC
    arrival_airport: JFK
    departure_date:


You:  No, it's good thanks


...


Fred: ================================ Human Message =================================

I am based in New York
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_eGIrJwmVJ5kBBxto73cdVRWO)
 Call ID: call_eGIrJwmVJ5kBBxto73cdVRWO
  Args:
    username: Jason
    preferences: warm places
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Jason: warm places
================================== Ai Message ==================================

Great, Jason! I have noted that you are interested in warm places. Where would you like to go for your holiday? Do you have any specific destination in mind?



You:  Dallas maybe?



Fred: ================================ Human Message =================================

Dallas maybe?
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_Bwc1hekCRTzNvZCrlHwad1n9)
 Call ID: call_Bwc1hekCRTzNvZCrlHwad1n9
  Args:
    username: Jason
    preferences: warm places
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Jason: warm places
================================== Ai Message ==================================

Great, Jason! Your preference for warm places has been saved. Now, let's look for flights from New York to Dallas. Let me find the best options for you.
Tool Calls:
  flight_search_tool (call_7R4sHRYDIAPmVzvSGwAHho4x)
 Call ID: call_7R4sHRYDIAPmVzvSGwAHho4x
  Args:
    departure_airport: JFK
    arrival_airport: DFW
    departure_date: 2022-11-15
================================


You:  quit



Fred: Goodbye! Have a great day!


In [41]:
def chat_with_agent(graph):
    print("\nWelcome! I'm Fred, your travel assistant. How can I help you today?")
    
    # Initialize conversation history
    messages = []
    
    while True:
        try:
            # Get user input
            user_input = input("\nYou: ")
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\nFred: Goodbye! Have a great day!")
                break
            
            # Add user message to history and create input state
            messages.append(("user", user_input))
            inputs = {"messages": messages}
            
            # Process through graph and get response
            print("\nFred:", end=" ")
            for s in graph.stream(inputs, stream_mode="values"):
                message = s["messages"][-1]
                if isinstance(message, tuple):
                    # Add to conversation history
                    messages.append(message)
                else:
                    message.pretty_print()
                    
        except KeyboardInterrupt:
            print("\n\nFred: Goodbye! Have a great day!")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try again or type 'exit' to quit.")


# Start the chat
# chat_with_agent(graph)

In [43]:
import gradio as gr
import os

def respond(message, history):
    """Process user message and return agent response"""
    # Convert history to the format expected by the agent
    messages = []
    for human, assistant in history:
        messages.append(("user", human))
        if assistant:  # Skip None responses
            messages.append(("assistant", assistant))
    
    # Add current message
    messages.append(("user", message))
    
    # Process through graph and get response
    inputs = {"messages": messages}
    
    try:
        for s in graph.stream(inputs, stream_mode="values", debug=True):
            message = s["messages"][-1]
            if isinstance(message, tuple):
                # Extract just the content from the tuple
                response = message[1]
            else:
                # Extract content from AIMessage or other message types
                response = message.content if hasattr(message, 'content') else str(message)
            yield response
    except Exception as e:
        yield f"Error: {str(e)}\nPlease try again or refresh the page."

# Update the Gradio interface
demo = gr.ChatInterface(
    respond,
    title="Fred the Travel Assistant",
    description="Ask Fred about your next trip!",
    examples=["Hi, I'm planning a trip"],
)

demo.launch()

/opt/anaconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', "Hi, I'm planning a trip")]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', "Hi, I'm planning a trip")]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content="Hi, I'm planning a trip", additional_kwargs={}, response_metadata={}, id='6d9a3dd8-0950-4465-91bf-af0228cfd9ab')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content="Hi, I'm planning a trip", additional_kwargs={}, response_metadata={}, id='6d9a3dd8-0950-4465-91bf-af0228cfd9ab')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content="Great to hear that you're planning a trip! Before we proceed, may I have your name, please?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_